In [3]:
%%writefile sum.cpp
#include<iostream>
#include<algorithm>
using namespace std;

constexpr int M = 2048;
constexpr int N = 2048;

double naive_sum(const double a[][N]){
    double sum = 0.0;
    for(int j = 0; j < N; ++j) {
        for(int i = 0; i < M; ++i)
            sum += a[i][j];
    }
    return sum;
}

double improved_sum(const double a[][N]) {
    double sum = 0.0;
    for(int i = 0; i < M; ++i)
        for(int j = 0; j < N; ++j)
            sum +=a[i][j];
    return sum;
}

int main() {
    static double a[M][N];
    fill_n(&a[0][0], M*N, 1.0 / (M*N));
    cout << naive_sum(a) << endl;
    static double b[M][N];
    fill_n(&b[0][0], M*N, 1.0 / (M*N));
    cout << improved_sum(b) << endl;
    return 0;
}

Overwriting sum.cpp


In [6]:
!g++ -Wall -O3 -g sum.cpp -o sum


In [8]:
!apt-get install valgrind

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
valgrind is already the newest version (1:3.18.1-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [9]:
!valgrind --tool=callgrind ./sum

==39968== Callgrind, a call-graph generating cache profiler
==39968== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==39968== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==39968== Command: ./sum
==39968== 
==39968== For interactive control, run 'callgrind_control -h'.
1
1
==39968== 
==39968== Events    : Ir
==39968== Collected : 50581278
==39968== 
==39968== I   refs:      50,581,278


In [12]:
!ls


Aula4.ipynb	     euclides-ingenuo.cpp  matriz.svg	sum
callgrind.out.39968  index.md		   sample_data	sum.cpp
euclides.dSYM	     matriz.png		   slides.pdf


In [15]:
!callgrind_annotate callgrind.out.39968 sum.cpp

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.39968' (creator: callgrind-3.18.1)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 10864277
Trigger: Program termination
Profiled target:  ./sum (PID 39968, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       99
Include dirs:     
User annotated:   sum.cpp
Auto-annotation:  on

--------------------------------------------------------------------------------
Ir                  
--------------------------------------------------------------------------------
50,581,278 (100.0%)  PROGRAM TOTALS

--------------------------------------------------------------------------------
Ir                   file:function
--------------------------------------------------------------------------------
31,475,723 (62.23%)  sum.cpp:main [/content/sum]
16,777,220 

In [16]:
%%writefile profile.cpp
#include <iostream>
#include <cmath>
#include <chrono>

void heavyCalculation() {
    for (int i = 0; i < 100000; ++i) {
        double result = std::sqrt(static_cast<double>(i));
    }
}

void intermediateFunction() {
    for (int i = 0; i < 1000; ++i) {
        heavyCalculation();
    }
}

void mainFunction() {
    for (int i = 0; i < 5; ++i) {
        intermediateFunction();
    }
}

int main() {
    auto start = std::chrono::high_resolution_clock::now();

    mainFunction();

    auto end = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - start);

    std::cout << "Time taken: " << duration.count() << " milliseconds" << std::endl;

    return 0;
}

Writing profile.cpp


In [17]:
!apt-get install graphviz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [18]:
!pip install gprof2dot

In [20]:
!gprof2dot -n30 -f callgrind callgrind.out.39968 | dot -Tsvg -o output.svg

In [23]:
!g++ -Wall -o0 -g profile.cpp -o profile

profile.cpp: In function ‘void heavyCalculation()’:
profile.cpp:7:16: warning: unused variable ‘result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
    7 |         double result = std::sqrt(static_cast<double>(i));
      |                ^~~~~~


In [28]:
!valgrind --tool=callgrind ./profile

==47894== Callgrind, a call-graph generating cache profiler
==47894== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==47894== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==47894== Command: ./profile
==47894== 
==47894== For interactive control, run 'callgrind_control -h'.
Time taken: 255619 milliseconds
==47894== 
==47894== Events    : Ir
==47894== Collected : 10002410933
==47894== 
==47894== I   refs:      10,002,410,933


In [29]:
!callgrind_annotate callgrind.out.39968 profile.cpp

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.39968' (creator: callgrind-3.18.1)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 10864277
Trigger: Program termination
Profiled target:  ./sum (PID 39968, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       99
Include dirs:     
User annotated:   profile.cpp
Auto-annotation:  on

--------------------------------------------------------------------------------
Ir                  
--------------------------------------------------------------------------------
50,581,278 (100.0%)  PROGRAM TOTALS

--------------------------------------------------------------------------------
Ir                   file:function
--------------------------------------------------------------------------------
31,475,723 (62.23%)  sum.cpp:main [/content/sum]
16,777,

In [31]:
!gprof2dot  -n0.1 -f callgrind callgrind.out.47894 | dot -Tsvg -o output.svg

In [32]:
!gprof2dot  -n0.1 -f callgrind callgrind.out.47894 | dot -Tsvg -o output3.svg

In [34]:
!g++ -g euclides-ingenuo.cpp -o euclides-ingenuo

In [36]:
!valgrind --tool=callgrind ./euclides-ingenuo

==53904== Callgrind, a call-graph generating cache profiler
==53904== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==53904== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==53904== Command: ./euclides-ingenuo
==53904== 
==53904== For interactive control, run 'callgrind_control -h'.
==53904== 
==53904== Process terminating with default action of signal 2 (SIGINT)
==53904==    at 0x4BB9992: read (read.c:26)
==53904==    by 0x4B31CB5: _IO_file_underflow@@GLIBC_2.2.5 (fileops.c:516)
==53904==    by 0x4B32E15: _IO_default_uflow (genops.c:362)
==53904==    by 0x496B8C0: __gnu_cxx::stdio_sync_filebuf<char, std::char_traits<char> >::underflow() (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.30)
==53904==    by 0x49799D5: std::istream::sentry::sentry(std::istream&, bool) (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.30)
==53904==    by 0x4979C31: std::istream::operator>>(int&) (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.30)
==53904==    by 0x1095